# getpairs.pynb

v0.2 vq

This iPython notebook demonstrates how to access DYFI and ShakeMap data from ComCat. We search the catalog for a list of events, download datasets for each event, and generate pairs of ShakeMap stations and DYFI aggregated cells that are co-located within a specified radius.

Procedure:
1. Required libraries
2. Searching ComCat and creating an event list
3. Downloading ShakeMap data
4. Downloading DYFI data
5. Finding ShakeMap-DYFI pairs
6. Looping through all events



### 1. Required libraries

In [1]:
from datetime import datetime
import json
import os
from glob import glob
import pandas as pd

from libcomcat.search import search,get_event_by_id
from esi_extern_openquake.geodetic import geodetic_distance as distance

### 2. Searching ComCat and creating an event list

Get entries from ComCat. For details on search parameters see:
https://github.com/usgs/libcomcat/blob/master/notebooks/Search.ipynb

In this example, we search for ShakeMaps in the Western United States in the first 3 months of 2010.

In [2]:
starttime=datetime(2010,1,1,0,0)
endtime=datetime(2010,3,1,0,0)
minlatitude=31.0
maxlatitude=50.0
minlongitude=-128.0
maxlongitude=-110.0
eventtype='earthquake'
events=search(starttime=starttime,
              endtime=endtime,
              minlatitude=minlatitude,
              maxlatitude=maxlatitude,
              minlongitude=minlongitude,
              maxlongitude=maxlongitude,
              eventtype=eventtype,
              producttype='shakemap')

This creates a list of events. Each member is a SummaryObject that contains event data and links to available products.

In [3]:
print('Got',len(events),'ShakeMap events.')
events[0]

Got 20 ShakeMap events.


uw10782693 2010-01-02 16:36:45.910000 (45.137,-120.956) 14.5 km M3.6

Now filter out events without DYFI products.

In [4]:
events=[e for e in events if e.hasProduct('dyfi')]
print('Got',len(events),'events with ShakeMap and DYFI products.')

Got 16 events with ShakeMap and DYFI products.


For the examples below, we select an event:

In [5]:
exampleEvent=events[3]
exampleEvent

nc71336726 2010-01-07 18:09:35.040000 (37.481,-121.799) 10.8 km M4.0

### 3. Downloading ShakeMap data

We access the data files for an event as JSON using libcomcat. To prevent unnecessary downloading and processing, we save the results to the directory `events` and in the dictionary `allevents`.

This function downloads the ShakeMap stations and turns into a Pandas dataframe. The columns are:

- Station ID
- Latitude
- Longitude
- Value (for ShakeMap: pga or other ground motion; for DYFI: CDI)
- Number of responses (for ShakeMap: always 1)

In [6]:
eventdir='./events'
os.makedirs(eventdir,exist_ok=True)

columns=['id','lat','lon','val','nresp']

allevents={ 'shakemap':{} }

shakemapParam='pga'
def getShakeMap(e):

    # Check if loaded previously
    if 'shakemap' in allevents and e.id in allevents['shakemap']:
        return allevents['shakemap'][e.id]
    
    datafile='stationlist.json'
    propTag=shakemapParam
        
    # Check if already saved to file
    savefile='%s/%s/%s' % (eventdir,e.id,datafile)
    fileglob=glob(savefile)
    if not fileglob:
        eventdata=get_event_by_id(e.id)
        product=eventdata.getProducts('shakemap')[0]
        os.makedirs('%s/%s' % (eventdir,e.id),exist_ok=True)
        url=product.getContent(datafile,savefile)
        print('Downloaded',url)
        
    with open(savefile,'r') as f:
        stations=json.load(f)['features']

    nresp=1
    results=[]
    for s in stations:
        val=s['properties'][propTag]
        if not val or val=='null':
            continue
            
        # Don't include macroseismic stations!
        id=s['id']
        if id[0:4]=='DYFI' or id[0:3]=='OBS':
            continue

        lon,lat=s['geometry']['coordinates']
        results.append([id,
                        round(float(lat),4),
                        round(float(lon),4),
                        float(val),
                        float(nresp)])

    results=pd.DataFrame(results,columns=columns)
    allevents['shakemap'][e.id]=results
    return results


shakemap=getShakeMap(exampleEvent)
print('This event has',len(shakemap),'ShakeMap stations.')
shakemap.head(5)


This event has 243 ShakeMap stations.


,id,lat,lon,val,nresp
0,CE.57031,37.3925,-121.9548,2.1197,1.0
1,CE.57064,37.5300,-121.9203,1.7975,1.0
2,CE.57370,37.2903,-121.7640,1.2628,1.0
3,CE.57384,37.5088,-121.9612,2.6369,1.0
4,CE.57431,37.5238,-121.9490,1.3403,1.0


### 4. Downloading DYFI data

Similarly, this function downloads the DYFI stations and turns into a DataFrame. In this example, we only use DYFI cells with at least 3 responses.

In [7]:
dyfiType='1km' 
minresp=3

allevents['dyfi']={}
def getDyfi(e):

    # Check if loaded previously
    if 'dyfi' in allevents and e.id in allevents['dyfi']:
        return allevents['dyfi'][e.id]
    
    datafile='dyfi_geo_%s.geojson' % dyfiType
    propTag='cdi'
        
    # Check if already saved to file
    savefile='%s/%s/%s' % (eventdir,e.id,datafile)
    fileglob=glob(savefile)
    if not fileglob:
        eventdata=get_event_by_id(e.id)
        product=eventdata.getProducts('dyfi')[0]
        os.makedirs('%s/%s' % (eventdir,e.id),exist_ok=True)
        url=product.getContent(datafile,savefile)
        print('Downloaded',url)
        
    with open(savefile,'r') as f:
        stations=json.load(f)['features']

    results=[]
    for s in stations:
        val=s['properties'][propTag]
        if not val or val=='null':
            continue
        nresp=s['properties']['nresp']
        if nresp<minresp:
            continue
        # Clean up UTM name
        id=s['properties']['name'].split('<br>')[0]

        # Get centroid of UTM cell
        lat=0; lon=0
        for pt in s['geometry']['coordinates'][0]:
            lat+=float(pt[0])
            lon+=float(pt[1])
        ncorners=len(s['geometry']['coordinates'][0])
        lat,lon=(round(lon/ncorners,4),round(lat/ncorners,4))

        results.append([id,lat,lon,val,nresp])

    results=pd.DataFrame(results,columns=columns)
    allevents['dyfi'][e.id]=results
    return results


dyfi=getDyfi(exampleEvent)
print('This event has',len(dyfi),'DYFI locations.')
dyfi[0:5]

This event has 173 DYFI locations.


,id,lat,lon,val,nresp
0,UTM:(10S 0545 4175 1000),37.7256,-122.4837,2.6,9
1,UTM:(10S 0546 4179 1000),37.7616,-122.4721,2.9,3
2,UTM:(10S 0549 4179 1000),37.7615,-122.4380,2.3,5
3,UTM:(10S 0549 4180 1000),37.7705,-122.4379,2.2,3
4,UTM:(10S 0549 4182 1000),37.7885,-122.4378,2.6,3


### 5. Finding ShakeMap-DYFI pairs

This function loops through ShakeMap stations and DYFI aggregated data for a given event, then find all pairs within a certain threshold. In this example, we only take pairs within 3 km of each other.

In [8]:
filter=3.0  # Threshold for pairing, in km
paircolumns=['Event ID',
             'ShakeMap station',
             shakemapParam,
             'DYFI cell',
             'cdi',
             'DYFI nresp']

def getPairs(event,filter=filter):
    pairs=[]
    # Convert dataframes to NumPy arrays for faster iteration
    shakemap=getShakeMap(event).to_numpy()
    dyfi=getDyfi(event).to_numpy()

    for s in shakemap:
        for d in dyfi:
            dist=distance(s[2],s[1],d[2],d[1])
            if dist<=filter:
                pairs.append([event.id,s[0],s[3],d[0],d[3],d[4]])

    pairs=pd.DataFrame(pairs,columns=paircolumns)
    return pairs
                
pairs=getPairs(exampleEvent,filter)
print('Found',len(pairs),'pairs within',filter,'km for this event.')
pairs

Found 336 pairs within 3.0 km for this event.


,Event ID,ShakeMap station,pga,DYFI cell,cdi,DYFI nresp
0,nc71336726,CE.57031,2.1197,UTM:(10S 0590 4138 1000),3.4,7
1,nc71336726,CE.57031,2.1197,UTM:(10S 0590 4139 1000),3.3,7
2,nc71336726,CE.57031,2.1197,UTM:(10S 0590 4140 1000),3.3,3
3,nc71336726,CE.57031,2.1197,UTM:(10S 0591 4136 1000),2.8,19
4,nc71336726,CE.57031,2.1197,UTM:(10S 0591 4137 1000),3.6,3
...,...,...,...,...,...,...
331,nc71336726,NP.1838,2.7006,UTM:(10S 0595 4140 1000),3.6,3
332,nc71336726,NP.1841,1.6527,UTM:(10S 0588 4133 1000),3.2,3
333,nc71336726,NP.1841,1.6527,UTM:(10S 0589 4137 1000),3.7,4
334,nc71336726,NP.1841,1.6527,UTM:(10S 0591 4136 1000),2.8,19


### 6. Looping through all events

Now we loop through all events to generate a dataset of all station pairs.

In [9]:
allpairs=pd.DataFrame([],columns=paircolumns)

for event in events:
    pairs=getPairs(event,filter=filter)
    if not pairs.empty:
        allpairs=pd.concat([allpairs,pairs],ignore_index=True)

allpairs

,Event ID,ShakeMap station,pga,DYFI cell,cdi,DYFI nresp
0,uu50393545,NP.2267,1.5944,UTM:(12S 0315 4170 1000),3.1,3
1,uu50393545,NP.2267,1.5944,UTM:(12S 0317 4172 1000),3.2,8
2,uu50393545,NP.7224,2.0741,UTM:(12S 0315 4170 1000),3.1,3
3,uu50393545,NP.7224,2.0741,UTM:(12S 0317 4172 1000),3.2,8
4,uu50393545,UU.CVH,3.6789,UTM:(12S 0318 4175 1000),2.8,3
...,...,...,...,...,...,...
442,ci10542525,CE.02140,0.9197,UTM:(11S 0502 3628 1000),3.4,3
443,ci10542525,CE.02140,0.9197,UTM:(11S 0503 3628 1000),2.0,3
444,ci10542525,CE.02143,1.4978,UTM:(11S 0500 3635 1000),3.2,4
445,ci10542525,CE.02143,1.4978,UTM:(11S 0502 3635 1000),2.7,3
